In [1]:
from dagster import asset, AssetExecutionContext, Definitions, define_asset_job, ScheduleDefinition
import requests
import pandas as pd
import io
import boto3
from datetime import datetime

# Garage S3 configuration
S3_ENDPOINT = 'http://localhost:3900'
S3_ACCESS_KEY = "GK904ad8d9d4e12205f574d8bb"
S3_SECRET_KEY = "9d9ec2f36a1787ba56675056312ac63719ef48fd198471bbcc722c036fcdae75"
S3_BUCKET = 'traffy-troffi'
S3_PREFIX = 'traffy-csvs'

@asset
def public_drive_csv_to_s3(context: AssetExecutionContext):
    """Download public CSV file from Google Drive and save it to Garage S3."""
    # Configuration
    file_id = "19QkF8i1my99gjbyHe7de_qZNwgrca6R5"  # The ID from the Drive URL

    # Create timestamp for the filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Log the process
    context.log.info(f"Starting download of public Google Drive CSV at {timestamp}")

    # Direct download URL for public files
    download_url = f"https://drive.google.com/uc?export=download&id={file_id}"

    # Get file metadata first (to get the name)
    metadata_url = f"https://drive.google.com/file/d/{file_id}/view"
    metadata_response = requests.get(metadata_url)

    # Extract filename from the response (this is a simple approach)
    # For more robust extraction, you might need additional parsing
    file_name = f"file_{file_id}.csv"  # Default name

    # Download the file content
    response = requests.get(download_url)
    if response.status_code == 200:
        file_content = response.content
    else:
        raise Exception(f"Failed to download file: {response.status_code}")

    # Create S3 client for Garage
    s3_client = boto3.client(
        's3',
        endpoint_url=S3_ENDPOINT,
        aws_access_key_id=S3_ACCESS_KEY,
        aws_secret_access_key=S3_SECRET_KEY
    )

    # Generate S3 key (path)
    s3_key = f"{S3_PREFIX}/traffy_{timestamp}.csv"

    # Upload to Garage S3
    s3_client.put_object(
        Bucket=S3_BUCKET,
        Key=s3_key,
        Body=file_content
    )

    context.log.info(f"Uploaded CSV to Garage S3 at {S3_BUCKET}/{s3_key}")

    # Parse CSV as dataframe
    df = pd.read_csv(io.BytesIO(file_content))

    return {
        "s3_bucket": S3_BUCKET,
        "s3_key": s3_key,
        "row_count": len(df)
    }